The first few cells set up phonetisaurus; they are adapted from the instructions in the git README.

In [ ]:
%%capture
!apt-get -y install git g++ autoconf-archive make libtool
# Python bindings
!apt-get -y install python-setuptools python-dev
# mitlm (to build a quick play model)
!apt-get -y install gfortran

In [ ]:
%%capture
!wget http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.2.tar.gz
!tar -xvzf openfst-1.6.2.tar.gz
%cd openfst-1.6.2
# Minimal configure, compatible with current defaults for Kaldi
!./configure --enable-static --enable-shared --enable-far --enable-ngram-fsts
!make -j 4
# Now wait a while...
!make install

In [17]:
import os
ldlibpath = os.environ['LD_LIBRARY_PATH']
#_STORED_LD = "/usr/local/nvidia/lib:/usr/local/nvidia/lib64"
newld = f'{ldlibpath}:/usr/local/lib:/usr/local/lib/fst'
os.environ['LD_LIBRARY_PATH']=newld
%env LD_LIBRARY_PATH

'/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib:/usr/local/lib/fst'

In [ ]:
%%capture
%cd /content
!git clone https://github.com/AdolfVonKleist/Phonetisaurus.git
%cd Phonetisaurus
!./configure
!make
!make install

In [36]:
%cd /content/

/content


We also need MITLM

In [ ]:
%%capture
!git clone https://github.com/mitlm/mitlm
%cd mitlm
!autoreconf -i
!./configure
!make
!make install

In [38]:
%cd /content

/content


The TIMIT dictionary is relatively clean, so there are only a few small changes that are needed for phonetisaurus.

In [75]:
!cat TIMITDIC.txt|grep -v '^;'|tr -d '/'|sed -e 's/  */ /g;s/~adj//;s/~v_past//;s/~v_pres//;s/~v//;s/~n//;' > TIMIT.clean

In [77]:
!phonetisaurus-align --input=TIMIT.clean --ofile=TIMIT.clean.corpus --seq1_del=false
# Train an n-gram model (5s-10s):
!estimate-ngram -o 8 -t TIMIT.clean.corpus -wl timit.arpa
# Convert to OpenFst format (10s-20s):
!phonetisaurus-arpa2wfst --lm=timit.arpa --ofile=timit.fst

GitRevision: 0.9.1
Loading input file: TIMIT.clean
Starting EM...
Finished first iter...
Iteration: 1 Change: 7.52312
Iteration: 2 Change: 0.00584698
Iteration: 3 Change: 0.00989723
Iteration: 4 Change: 0.00993443
Iteration: 5 Change: 0.00851822
Iteration: 6 Change: 0.00636768
Iteration: 7 Change: 0.00507736
Iteration: 8 Change: 0.00345325
Iteration: 9 Change: 0.00275803
Iteration: 10 Change: 0.00178242
Iteration: 11 Change: 0.0011425
Last iteration: 
0.001	Loading corpus TIMIT.clean.corpus...
0.107	Smoothing[1] = ModKN
0.107	Smoothing[2] = ModKN
0.107	Smoothing[3] = ModKN
0.107	Smoothing[4] = ModKN
0.107	Smoothing[5] = ModKN
0.107	Smoothing[6] = ModKN
0.107	Smoothing[7] = ModKN
0.107	Smoothing[8] = ModKN
0.107	Set smoothing algorithms...
0.107	Y 8.000000e-01
0.107	Y 5.139752e-01
0.107	Y 6.063855e-01
0.107	Y 7.198564e-01
0.108	Y 8.121408e-01
0.109	Y 8.843038e-01
0.110	Y 9.301026e-01
0.111	Y 9.179621e-01
0.111	Estimating full n-gram model...
0.115	Saving LM to timit.arpa...
GitRevision:

In [78]:
!cat TIMIT.clean|cut -d ' ' -f 1|head -n 50 > TIMIT.list

In [ ]:
!phonetisaurus-apply -v --model timit.fst --word_list TIMIT.list